# Load data
This code loads the data for 1 subject with the help of methods defined in data_handler.ipynb.
Also prints distribution of the split just to make sure it's stratified.

In [24]:
import ipynb
import torch
from ipynb.fs.full.data_handler import load_subject, extract_labels, extract_action_interval, split_data, split_info, to_device, get_innerspeech
from ipynb.fs.full.train_model import train_model, accuracy_check

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

data, description = load_subject(1)
data, description = get_innerspeech(data, description)
labels = extract_labels(description)
#data_interval = extract_action_interval(data)

train_data, val_data, test_data, train_labels, val_labels, test_labels = split_data(data, labels)

train_data, val_data, test_data, train_labels, val_labels, test_labels = to_device(train_data, val_data, test_data, train_labels, val_labels, test_labels, device)
split_info(train_data, val_data, test_data, train_labels, val_labels, test_labels)

[39, 39, 40, 40] 
 [6, 6, 5, 5] 
 [5, 5, 5, 5]
24.68354430379747 %  24.68354430379747 %  25.31645569620253 %  25.31645569620253 % 
27.27272727272727 %  27.27272727272727 %  22.727272727272727 %  22.727272727272727 % 
25.0 %  25.0 %  25.0 %  25.0 % 


# EEGNet
Implementation of EEGNetfrom (Lawhern et. al.) for the use on "Thinking out loud" dataset.

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim


class EEGNet(nn.Module):
    def __init__(self, interval = "full", dropout = 0.5):
        super(EEGNet, self).__init__()
        self.f1 = 16
        self.d = 4
        self.f2 = self.d*self.f1

        # Block 1
        self.conv1 = nn.Conv2d(in_channels=1, out_channels = self.f1, kernel_size = (1, 128), padding = 0, bias = False) # 128 = half sampling rate, reduces to 2Hz, 16 is probably F1, the number of filters (output channels)
        self.batchnorm1 = nn.BatchNorm2d(self.f1, False)

        self.conv2 = nn.Conv2d(in_channels = self.f1, out_channels = self.f1*self.d, kernel_size = (128, 1), padding = 0, bias = False) # row, column] 128 = number of sensors for the depth wise convolution
        self.batchnorm2 = nn.BatchNorm2d(self.f1*self.d, False) # Not sure this is the right number of features.
        # Elu
        self.elu = nn.ELU()
        # Dropout
        self.dropout = nn.Dropout(dropout)

        self.avgPool1 = nn.AvgPool2d((1,4)) # Reduces sampling rate to 64hz

        # Block 2

        # Separable convolution
        self.depthwise = nn.Conv2d(in_channels = self.f1*self.d, out_channels = self.f1*self.d, kernel_size = (1, 32),
                        bias=False, groups = self.f1*self.d, padding = (0, 16//2)) # Captures 500ms of data at sampling rate 64Hz (32 Hz is 500ms of that)
        self.pointwise = nn.Conv2d(in_channels = self.f1*self.d, out_channels = self.f2, kernel_size = 1) # This results in f2 channels of the output here.
        
        self.batchnorm3 = nn.BatchNorm2d(self.f2, False)

        self.avgPool2 = nn.AvgPool2d((1,8))
        
        # Fully connected classifier
        if interval == "action":
            self.classifier = nn.Linear(896,4, bias = False)
        elif interval == "full":
            self.classifier = nn.Linear(1920,4, bias = False)
        self.softmax = nn.LogSoftmax(dim = 1)


    def forward(self, x): # Dropout 0.5 dropout for within-subject and 0.25 for cross-subject
        # Block 1
        res = self.conv1(x)
        res = self.batchnorm1(res)
        res = self.conv2(res)
        res = torch.renorm(res, p=2, dim=0, maxnorm=1)
        res = self.batchnorm2(res)
        #res = F.dropout(0.25)
        res = self.elu(res)
        res = self.avgPool1(res)
        res = self.dropout(res)
        # Block 2
        res = self.depthwise(res)
        res = self.pointwise(res)
        res = self.batchnorm3(res)
        res = self.elu(res)
        res = self.avgPool2(res)
        res = self.dropout(res)
        # Classifier
        res = torch.flatten(res, start_dim=1)
        res = self.classifier(res)
        res = self.softmax(res)
        return res


## Testing code
Code for testing the EEGNet implementation

In [26]:
network = EEGNet(interval = "full").float().to(device)
accuracy_check(network, train_data, train_labels)

print("####### TRAINING #######")
op = optim.Adam(params = network.parameters(), lr = 0.0001)
lossf = nn.NLLLoss()
train_model(network, train_data = train_data, train_labels = train_labels, val_data = val_data, val_labels = val_labels, epochs = 50, batch_size = 10, loss_func = lossf, optimizer = op)

accuracy_check(network, train_data, train_labels)

ACCURACY: 0.2468354430379747
####### TRAINING #######
Epoch	 train loss	 validation loss
0 	  1.44306271870931 	 1.3870223760604858
1 	  1.296536636352539 	 1.3874619007110596
2 	  1.2237531026204427 	 1.3880420923233032
3 	  1.1862232208251953 	 1.3893556594848633
4 	  1.099804433186849 	 1.3935351371765137
5 	  1.0627496719360352 	 1.4082878828048706
6 	  1.0112250645955403 	 1.4567588567733765
7 	  0.9774152119954427 	 1.5460773706436157
8 	  0.9034236907958985 	 1.5744907855987549
9 	  0.8308194478352865 	 1.5913726091384888
10 	  0.8049777348836263 	 1.6063998937606812
11 	  0.7355347951253255 	 1.598737359046936
12 	  0.7172224680582683 	 1.6007953882217407
13 	  0.6638588587443034 	 1.5988967418670654
14 	  0.6114103317260742 	 1.6298378705978394
15 	  0.5432328542073568 	 1.6007555723190308
16 	  0.5073741912841797 	 1.5796517133712769
17 	  0.4546563784281413 	 1.6207469701766968
18 	  0.4339114824930827 	 1.6759943962097168
19 	  0.38131853739420574 	 1.7006396055221558
20 	 

In [27]:
accuracy_check(network, train_data, train_labels)
accuracy_check(network, val_data, val_labels)
accuracy_check(network, test_data, test_labels)

ACCURACY: 0.9683544303797469
ACCURACY: 0.36363636363636365
ACCURACY: 0.25
